In [18]:
import ujson
import glob
import os
import re
import pandas as pd
from nltk.parse.corenlp import CoreNLPParser

parser = CoreNLPParser(url='http://localhost:9010', tagtype='pos')

In [2]:
# load the non-dreprecated java doc caveat sentences of methods (parameters or exception level)
caveat_files_dir = './output/java_12_spec_caveat_sentences_revised/'
method_caveats = []
parameter_caveats = []
exception_caveats = []

files = sorted(glob.glob(caveat_files_dir + '*.json'))
for file in files:
    with open(file) as f:
        arr = ujson.load(f)
        full_class_name = os.path.splitext(os.path.basename(file))[0]
        simple_class_name = full_class_name.split('.')[-1]
        
        for caveat in arr:
            if not caveat['deprecated'] and 'name' in caveat:
                
                # collect the name of parameters
                collected = False
                parameters = []
                for misc_obj in caveat['caveat_misc']:
                    if misc_obj['name'] == 'Parameters:':
                        for obj in misc_obj['list']:
                            parameters.append(obj['parameter'])
                        collected = True
                        break
                        
                for sentence in caveat['sentences']:
                    method_caveats.append({
                                    'obj': '',
                                    'simple_class_name': simple_class_name,
                                    'full_class_name': full_class_name,
                                    'api': caveat['name'],
                                    'signature': caveat['signature'],
                                    'sentence': sentence,
                                    'parameters': parameters,
                                    'type': 'method'
                                })

                if collected:
                    # add all parameter and exception level sentences
                    for misc_obj in caveat['caveat_misc']:
                        if misc_obj['name'] in ['Parameters:', 'Throws:']:
                            for obj in misc_obj['list']:
                                for misc_sentence in obj['sentences']:
                                    if misc_obj['name'] == 'Parameters:':
                                        parameter_caveats.append({
                                            'obj': obj['parameter'],
                                            'simple_class_name': simple_class_name,
                                            'full_class_name': full_class_name,
                                            'api': caveat['name'],
                                            'signature': caveat['signature'],
                                            'sentence': misc_sentence,
                                            'parameters': parameters,
                                            'type': 'parameter'
                                        })
                                    else:
                                        exception_caveats.append({
                                            'obj': obj['exception'],
                                            'simple_class_name': simple_class_name,
                                            'full_class_name': full_class_name,
                                            'api': caveat['name'],
                                            'signature': caveat['signature'],
                                            'sentence': misc_sentence,
                                            'parameters': parameters,
                                            'type': 'exception'
                                        })
                                        
method_caveat_df = pd.DataFrame(method_caveats)
parameter_caveat_df = pd.DataFrame(parameter_caveats)
exception_caveat_df = pd.DataFrame(exception_caveats)

In [3]:
def filter_by_pattern(sentence, patterns, lowercase):
    if lowercase:
        sentence = sentence.lower()
    if sentence:
        for pattern in patterns:
            matches = re.search(pattern, sentence)
            if matches:
                return True
    
    return False

# Regex patterns for filtering
exception_not_null_patterns = [
#     '(be|equal|equals|is|are) null',
#     'be (equal|equivalent) to null',
#     'non-null',
     'null'
]

exception_range_limitation_patterns = [
    r'<|>|=',
    r'equal|equal to|equivalent to|illegal value| is (nan|infinite|empty)',
    r'\b(less|smaller|greater|larger)\b',
    r'\b(range|negative|positive|non-negative|non-positive)\b'
]

exception_type_restriction_patterns = [
    r'is( not)? an? [A-Z][a-z]+([A-Za-z_0-9\.]*)*',
    r'instance of|return type'
]

parameter_not_null_patterns = [
    r'not( be)? null',
    r'non-null',
]

parameter_range_limitation_patterns = [
     r'<|>|=',
     r'(less|smaller|greater|larger) than',
     r'negative|positive|non-negative|non-positive'
]

def exception_not_null_filter(sentence):
    return filter_by_pattern(sentence, exception_not_null_patterns, lowercase=True)

def exception_range_limitation_filter(sentence):
    return filter_by_pattern(sentence, exception_range_limitation_patterns, lowercase=True)

def exception_type_restriction_filter(sentence):
    return filter_by_pattern(sentence, exception_type_restriction_patterns, lowercase=False)

def parameter_not_null_filter(sentence):
    return filter_by_pattern(sentence, parameter_not_null_patterns, lowercase=True)

def parameter_range_limitation_filter(sentence):
    return filter_by_pattern(sentence, parameter_range_limitation_patterns, lowercase=True)

In [4]:
exception_filters = {
    'not_null': (exception_not_null_filter, 'NullnessNotAllowed'),
    'range': (exception_range_limitation_filter, 'RangeLimitation'),
    'type': (exception_type_restriction_filter, 'TypeRestriction')
}

parameter_filters = {
    'not_null': (parameter_not_null_filter, 'NullnessNotAllowed'),
    'range': (parameter_range_limitation_filter, 'RangeLimitation')
}

def analyse_labelled_results(file, filters):
    with open(file) as f:
        counts = {}
        c = 0
        for line in f:
            obj = ujson.loads(line)

            if obj['labels']:
                c += 1
                if c > 384:
                    break

                for label in obj['labels']:
                    if not label in counts:
                        counts[label] = 1
                    else:
                        counts[label] += 1
        print(counts)

        for key in filters:
            print('Filter results for {}'.format(key))
            correct, retrieved = 0, 0
            c = 0
            f.seek(0)
            for line in f:
                obj = ujson.loads(line)

                if obj['labels']:
                    c += 1
                    if c > 384:
                        break

                    if filters[key][0](obj['text']):
                        retrieved += 1
                        if filters[key][1] in obj['labels']:
                            correct += 1
#                         else:
#                             print(obj)
#                     elif filters[key][1] in obj['labels']:
#                         print('MISS')
#                         print(obj)

            print('Correct: {}'.format(correct))
            print('Retrieved: {}\n'.format(retrieved))
  
# Test filtering rules
print('Exception caveat sentences')
analyse_labelled_results('./labelled_data/labelled_exception_full.jsonl', exception_filters)
print('Parameter caveat sentences')
analyse_labelled_results('./labelled_data/labelled_parameter_full.jsonl', parameter_filters)

Exception caveat sentences
{'Ambiguous': 256, 'NullnessNotAllowed': 92, 'RangeLimitation': 27, 'TypeRestriction': 9, 'Dependent': 9}
Filter results for not_null
Correct: 92
Retrieved: 92

Filter results for range
Correct: 27
Retrieved: 35

Filter results for type
Correct: 9
Retrieved: 10

Parameter caveat sentences
{'NullnessNotAllowed': 36, 'Ambiguous': 222, 'ExpectedValue': 3, 'RangeLimitation': 3}
Filter results for not_null
Correct: 36
Retrieved: 37

Filter results for range
Correct: 3
Retrieved: 7



In [29]:
method_caveat_df = pd.DataFrame(method_caveats)
parameter_caveat_df = pd.DataFrame(parameter_caveats)
exception_caveat_df = pd.DataFrame(exception_caveats)

def get_unique_filtered_df(df, filter_func):
    filtered_df = df[df['sentence'].apply(filter_func)]
    unique_df = filtered_df.drop_duplicates('sentence').sample(frac=1)
    print('Filtered results: {}'.format(len(filtered_df.index)))
    print('Unique results: {}\n'.format(len(unique_df.index)))
    return unique_df

# Not null exception
not_null_exception_df = get_unique_filtered_df(exception_caveat_df, exception_not_null_filter)

# Not null paramater
not_null_parameter_df = get_unique_filtered_df(parameter_caveat_df, parameter_not_null_filter)

# Range limitation exception
range_limit_exception_df = get_unique_filtered_df(exception_caveat_df, exception_range_limitation_filter)

# Range limitation parameter
range_limit_parameter_df = get_unique_filtered_df(parameter_caveat_df, parameter_range_limitation_filter)

# Type restriction exception
type_restrict_exception_df = get_unique_filtered_df(exception_caveat_df, exception_type_restriction_filter)

def create_annotation_file(df, size, out_path, include_obj=False):
    with open(out_path, 'w+') as out_f:
        c = 0
        for i in df.index:
            c += 1
            if c == 1:
                out_f.write('-----------------------------')
            
            if c > size:
                break
            
            out_f.write(str(i) + '\n')
            out_f.write(df.loc[i, 'signature'] + '\n')
            if include_obj:
                out_f.write(df.loc[i,'obj'] + '\n')
            out_f.write(df.loc[i, 'sentence'] + '\n\n')
            out_f.write('-----------------------------')
            

# create_annotation_file(type_restrict_exception_df, 100, './output/labelled_caveat_rules/type_restrict_exception.txt')
# create_annotation_file(not_null_exception_df, 100, './output/labelled_caveat_rules/not_null_exception.txt')
# create_annotation_file(not_null_parameter_df, 100, './output/labelled_caveat_rules/not_null_parameter.txt', True)
# create_annotation_file(range_limit_exception_df, 100, './output/labelled_caveat_rules/range_limit_exception.txt')
# create_annotation_file(range_limit_parameter_df, 100, './output/labelled_caveat_rules/range_limit_parameter.txt', True)

Filtered results: 3844
Unique results: 1447

Filtered results: 1202
Unique results: 495

Filtered results: 1946
Unique results: 834

Filtered results: 335
Unique results: 193

Filtered results: 208
Unique results: 149



In [30]:
def read_labelled_rule_results(path, parser, rule_start=3, offset=4):
    with open(path) as f:
        lines = f.readlines()
        
        indices = [re.sub("[^0-9]", "", line) for line in lines[::offset]]
        rules = [line.strip() for line in lines[rule_start::offset]]
        
        if len(indices) > len(rules):
            indices = indices[:len(rules)]
        
        results = {}
        for i, val in enumerate(indices):
            results[val] = parser(rules[i])
            
        return results
    
def or_parser(expression):
    if expression == '':
        return None
    return expression.split('||')

def type_parser(expression):
    if expression == '':
        return None
    
    expressions = expression.split(' & ')
    results = []
    for exp in expressions:
        components = exp.split(' -> ')
        types = components[1].split('||')
        negated = components[0][0] == '!'
        
        obj = components[0]
        if components[0][0] == '!':
            obj = components[0][1:]
        results.append({'obj': obj, 'types': types, 'negated':negated})
    
    return results
        
not_null_exception_results = read_labelled_rule_results('./labelled_data/not_null_exception.txt', or_parser)
print(len([x for x in not_null_exception_results if not_null_exception_results[x]]))

not_null_parameter_results = read_labelled_rule_results('./labelled_data/not_null_parameter.txt', or_parser, 4, 5)
print(len([x for x in not_null_parameter_results if not_null_parameter_results[x]]))

range_limit_exception_results = read_labelled_rule_results('./labelled_data/range_limit_exception.txt', or_parser)
print(len([x for x in range_limit_exception_results if range_limit_exception_results[x]]))

range_limit_parameter_results = read_labelled_rule_results('./labelled_data/range_limit_parameter.txt', or_parser, 4, 5)
print(len([x for x in range_limit_parameter_results if range_limit_parameter_results[x]]))

type_restrict_exception_results = read_labelled_rule_results('./labelled_data/type_restrict_exception.txt', type_parser)
print(len([x for x in type_restrict_exception_results if type_restrict_exception_results[x]]))

87
90
72
49
33


In [281]:
def get_type_rules(api_caveat_obj):
    skip_patterns = [
        r'subclass|at the specified',
        r'any .*element of .* is( not)? an?',
        r'and ([a-z]\w+) is( not)? an? ((?:[A-Z]\w+(?:, )?)+)(?:or(?: a)? ([A-Z]\w+))?',
        r' ([a-z]\w+) is( not)? an? ((?:[A-Z]\w+(?:, )?)+)(?:or(?: a)? ([A-Z]\w+))? and'
    ]
    for pattern in skip_patterns:
        if re.search(pattern, api_caveat_obj['sentence']):
            return
    matches = re.search(r' ([a-z]\w+) is( not)? a Class that implements interface ([A-Z]\w+)', api_caveat_obj['sentence'])
    if matches:
        results = []
        negated = matches.group(1) != None
        results.append({'obj': matches.group(0), 'types': [matches.group(2)], 'negated': negated})
    
    else:
        matches = re.findall(r' ([a-z]\w+) is( not)? an? ((?:[A-Z]\w+,? ?)+)(?:or ([A-Z]\w+))?', api_caveat_obj['sentence'])
        if matches:
            results = []
            for match in matches:
                if match[0] in api_caveat_obj['parameters']:
                    types = [e.strip() for e in match[2].split(', ') if len(e) > 0]
                    if match[3]:
                        types.append(match[3])
                    # only look at class names (first char is uppercase)
                    types = [t for t in types if t[0].isupper()]
                    if len(types) == 0:
                        continue

                    negated = match[1] == '' # not we check the reversed negation
                    results.append({'obj': match[0], 'types': types, 'negated': negated})

            return results
    
def is_not_null_parameter_caveat(api_caveat_obj):
    sentence = api_caveat_obj['sentence'].lower()
    
    # skip conditional
    if 'if ' in sentence:
        return False
    
    patterns = [
        r'not( be)? null',
        r'non-null'
    ]
    
    for pattern in patterns:
        if re.search(pattern, sentence):
            return True
    return False

def get_not_null_exception_rules(api_caveat_obj):
    # skip additional condition patterns
    skip_patterns = [
        r' and .* is null',
        r' is null and',
        r' any of the arguments',
        r' contains null element'
    ]
    
    types = []
    
    for pattern in skip_patterns:
        if re.search(pattern, api_caveat_obj['sentence']):
            return types
    
    # assume the caveat is about single parameters
    if len(api_caveat_obj['parameters']) == 1 and re.search(' null', api_caveat_obj['sentence']):
        return api_caveat_obj['parameters']
    
    match = re.search(r'([Ii]f|[Ww]hen) (.*) null', api_caveat_obj['sentence'])
    if match:
        tokens = re.sub(', ', ' ', match.group(2)).split()
        types = [t for t in tokens if t in api_caveat_obj['parameters']]
    return set(types)

def get_range_limitation_exception_rules(api_caveat_obj):   
    pass

In [789]:
def normalise_sentence(sentence, parameters):      
    # variable substitutions
    variable_subs = [
        r'\W(-?[0-9]+(?:,[0-9]+)*(?:(?:\.[0-9]+)?[a-z]*))', # specific numeric value
        r'\W("[^"]+")', # simple string
        r'([A-Za-z_]+[A-Za-z_0-9]*\.[A-Za-z_][A-Za-z_0-9]*\(\))', # class method
        r'((^(java\.|javax\.|org\.))?([A-Za-z_]\w*\.)+\w+)', # member value of object/Class
        r'(([A-Za-z_][A-Za-z_0-9])*\(\))' # standalone methods (empty param list only)
    ]
        
    # list of expression substitutions for logical phrases, note ordering is used
    worded_subs = [
        (r'(not? less than)', '>='),
        (r'(not? greater than)', '<='),
        (r'(greater than or equal to)', '>='),
        (r'(less than or equal to)', '<='),
        (r'(less than)', '<'),
        (r'(greater than)', '>'),
        (r'((is|are)? negative)', '< 0'),
        (r'((is|are)? positive)', '< 0'),
        (r'(not equal to)', '!='),
        (r'( is not )', '!='),
        (r'(equal to)', '==')
    ]
    
    # normalise all equality phrases
    for t in worded_subs:
        match = re.search(t[0], sentence)
        while match:
            sentence = re.sub(match.group(1), t[1], sentence)
            match = re.search(t[0], sentence)
            
        variable_prefix = 'VAR_'
    variable_counter = 'a'
    variable_dict = {}
    
    # normalise all API call parameters
    for parameter in parameters:
        print(parameter)
        key = variable_prefix + str(variable_counter)
        variable_dict[key] = parameter
        sentence = re.sub(r'\b'+parameter+r'\b', key, sentence)
        variable_counter = chr(ord(variable_counter) + 1)
    
    # normalise all variables that match predefined expression patterns
    for pattern in variable_subs:
        match = re.search(pattern, sentence)
        while match:
            key = variable_prefix + str(variable_counter)
            variable_dict[key] = match.group(1)
            next_pattern = re.escape(match.group(1))
            sentence = re.sub(next_pattern, key, sentence)
            variable_counter = chr(ord(variable_counter) + 1)
            
            match = re.search(pattern, sentence)

    # locate enumerations (lists of items)
    list_dict = {}
    list_counter = 0
    list_prefix = 'LIST_'
    pattern = '\W((\s*[A-Za-z0-9_@]+\s*)(,\s*[A-Za-z0-9_@]+\s*)+,?\s*or\s*[A-Za-z0-9_@]+)\W'
    match = re.search(pattern, sentence)

    while match:
        key = list_prefix + str(list_counter)
        sentence = re.sub(match.group(1), key, sentence)
        list_dict[key] = match.group(1)
        
        match = re.search(pattern, sentence)
        list_counter += 1
    
    expressions_dict = {}
    expression_prefix = 'EXPR_'
    expression_counter = 0
    start_index = 0
    bracket_counter = 0
    for i, c in enumerate(sentence):
        if c == '(': 
            bracket_counter += 1
        elif c == ')':
            bracket_counter -= 1
            # TODO: malformed sentences
            if bracket_counter == 0:
                substr = sentence[start_index:i]
                start_index = i + 1
                
                if re.search(r'-|\+|\*|/|<|<=|>|>=|==|!=|&&|\|\||\.\.', substr):
                    escaped = re.escape(substr)
                    key = expression_prefix + str(expression_counter)
                    expressions_dict[key] = substr
                    sentence = re.sub(escaped, key, sentence)
                    expression_counter += 1
    
    # remove spaces around operators if adjacent to valid variables/digits
    match = re.search(r'([A-Za-z_\(\)][A-Za-z0-9_\.\(\)]*)\s+(-|\+|\*|/|<|<=|>|>=|==|!=|&&|\|\||&|\.\.)\s+([A-Za-z_\(\)][A-Za-z0-9_\.\(\)]+)', sentence)
    while match and not match in seen_expressions:
        print(match.groups())
        sub = match.group(2)
        if not (match.group(1).startswith('VAR_') or match.group(1).startswith('LIST_')):
            sub = ' ' + sub
        if not (match.group(3).startswith('VAR_') or match.group(3).startswith('LIST_')):
            sub = sub + ' '
         
        sub = match.group(1) + sub + match.group(3)
        escaped = re.escape(match.group(0))
        print(sentence)
        sentence = re.sub(escaped, sub, sentence)
        seen_expressions.add(match)
        
        match = re.search(r'([A-Za-z_\(\)][A-Za-z0-9_\.\(\)]*)\s+(-|\+|\*|/|<|<=|>|>=|==|!=|&&|\|\||&|\.\.)\s+([A-Za-z_\(\)][A-Za-z0-9_\.\(\)]+)', sentence)
#         if len(seen_expressions) > 2:
#             break
    return sentence, variable_dict, list_dict 

In [795]:
sentence = "if iv is null or (iv.length - offset < 2 * (wordSize / 8))"
expressions_dict = {}
expression_prefix = 'EXPR_'
expression_counter = 0
start_index = 0
bracket_counter = 0
old_len = 0
first = True

while first or len(expressions_dict) != old_len:
    first = False
    old_len = len(expressions_dict)
    for i, c in enumerate(sentence):
        if c == '(': 
            bracket_counter += 1
            start_index = i
        elif c == ')':
            bracket_counter -= 1
            # TODO: malformed sentences
            if bracket_counter == 0:
                substr = sentence[start_index:i]
                start_index = i + 1

                if re.search(r'-|\+|\*|/|<|<=|>|>=|==|!=|&&|\|\||\.\.', substr):
                    escaped = re.escape(substr)
                    key = expression_prefix + str(expression_counter)
                    expressions_dict[key] = substr
                    sentence = re.sub(escaped, key, sentence)
                    expression_counter += 1
    
print(sentence)
print(expressions_dict)

if iv is null or EXPR_1)
{'EXPR_0': '(wordSize / 8)', 'EXPR_1': '(iv.length - offset < 2 * EXPR_0'}


In [790]:
test = "if iv is null or (iv.length - offset < 2 * (wordSize / 8))"
print(normalise_sentence(test, ['version', 'rounds', 'wordSize', 'offset']))

version
rounds
wordSize
offset
if iv is null or (VAR_g - VAR_d < VAR_e * (VAR_c / VAR_f))
('(VAR_g', '-', 'VAR_d')
if iv is null or (VAR_g - VAR_d < VAR_e * (VAR_c / VAR_f))
('VAR_d', '<', 'VAR_e')
if iv is null or (VAR_g -VAR_d < VAR_e * (VAR_c / VAR_f))
('VAR_e', '*', '(VAR_c')
if iv is null or (VAR_g -VAR_d<VAR_e * (VAR_c / VAR_f))
('(VAR_c', '/', 'VAR_f))')
if iv is null or (VAR_g -VAR_d<VAR_e* (VAR_c / VAR_f))
('if iv is null or (VAR_g -VAR_d<VAR_e* (VAR_c /VAR_f))', {'VAR_a': 'version', 'VAR_b': 'rounds', 'VAR_c': 'wordSize', 'VAR_d': 'offset', 'VAR_e': '2', 'VAR_f': '8', 'VAR_g': 'iv.length'}, {})


In [782]:
k = 10935
# sent = 'a != b and b < 1'
# expressions_dict = {}
# normalize_sentence_expressions(sent)
get_range_limitation_exception_rules(range_limit_exception_df.loc[k].to_dict())
# print(range_limit_exception_results[str(k)])

({'EXPR_0': ('iv.length - offset', 'iv.length', 'offset'), 'EXPR_1': ('2 * (wordSize', '2', '(wordSize'), 'EXPR_2': ('0 < EXPR', '0', 'EXPR', '')}, 'if iv is null or (EXPR_EXPR_2_1 / 8))')
[[('if', 'IN'), ('iv', 'NN'), ('is', 'VBZ'), ('null', 'JJ')], [('iv.length', 'JJ'), ('-', ':'), ('offset', 'VBN'), ('2', 'CD'), ('wordSize', 'NN'), ('8', 'CD')]]


In [102]:
k = 12403
print(get_not_null_exception_rules(not_null_exception_df.loc[k].to_dict()))
print(not_null_exception_results[str(k)])

{'type', 'source', 'connectionId'}
['type', 'source', 'connectionId']


In [678]:
print(get_type_rules(type_restrict_exception_df.loc[12403].to_dict()))
print(type_restrict_exception_results['1762'])

[{'obj': 'src', 'types': ['IndexColorModel'], 'negated': True}]
None


In [684]:
# evaluate type rules generation via regex
retrieved, correct, total = 0, 0, 0
for key in type_restrict_exception_results:
    index = int(key)
    caveat_obj = type_restrict_exception_df.loc[index].to_dict()
    
    rules = get_type_rules(caveat_obj)
    if rules:
        retrieved += 1
        
        if not type_restrict_exception_results[key]:
            print('MISFIND')
            print(index)
            continue
        
        is_correct = True
        if not type_restrict_exception_results[key]:
            is_correct = False
        
        has_match = False
        for rule in rules:
            match = [e for e in type_restrict_exception_results[key] if e['obj'] == rule['obj']]
            if len(match) > 0 and match[0]['types'] == rule['types'] and match[0]['negated'] == rule['negated']:
                has_match = True
                break
        
        if is_correct and has_match:
            print(index)
            correct += 1
            
    if type_restrict_exception_results[key]:
        total += 1
print('retrieved:{}, correct:{}, total: {}'.format(retrieved, correct, total))

16414
16423
7928
5174
8009
16388
16411
7990
10139
14367
12594
12676
16398
12607
MISFIND
1762
retrieved:15, correct:14, total: 33


In [727]:
# evaluate not null rules (parameter sentences) generation via regex
retrieved, correct, total = 0, 0, 0
for key in not_null_parameter_results:
    index = int(key)
    caveat_obj = not_null_parameter_df.loc[index].to_dict()
    
    rule = is_not_null_parameter_caveat(caveat_obj)
    if rule:
        retrieved += 1
        
        if not not_null_parameter_results[key]:
            print(index)
            continue
        
        correct += 1
            
    if not_null_parameter_results[key]:
        if not rule:
            print(index)
        total += 1
print('retrieved:{}, correct:{}, total: {}'.format(retrieved, correct, total))

4227
retrieved:91, correct:90, total: 90


In [840]:
# evaluate not null rules (exception sentences) generation via regex
retrieved, correct, total = 0, 0, 0

for key in not_null_exception_results:
    index = int(key)
    caveat_obj = not_null_exception_df.loc[index].to_dict()
    
    rules = get_not_null_exception_rules(caveat_obj)
    if rules:
        retrieved += 1
        
        if not not_null_exception_results[key] or set(not_null_exception_results[key]) != set(rules):
            print(index)
            continue
        
        correct += 1
            
    if not_null_exception_results[key]:
        if not rules:
            print(index)
        total += 1
print('retrieved:{}, correct:{}, total: {}'.format(retrieved, correct, total))

3078
8702
12013
3994
8651
12091
12587
12519
3112
14946
949
8146
15643
3364
12088
retrieved:81, correct:75, total: 84


In [517]:
def preprocess(sentence):
    if sentence[-1] == '.':
        sentence = sentence[:-1]
    
    sentence = re.sub(r'[^\w\. -]', '', sentence)
    # remove brackets surrounding text
    matches = re.findall(r' ([^)])', sentence)
    for match in matches:
        sentence = re.sub(match, match[0], sentence)
    return sentence

def tokenize(sentence):    
    return sentence.split()

test = 'if value is < @VAR_0 or > @VAR_1, and @VAR_3 > @VAR_2'
processed_text = tokenize(preprocess(test))
# print(parser.tag(processed_text))
print(next(next(parser.parse_sents([processed_text]))).pretty_print())
# next(parser.raw_parse('if number is null or not an instance of Number')).pretty_print()

     ROOT                                                                             
      |                                                                                
     SBAR                                                                             
  ____|________                                                                        
 |             S                                                                      
 |     ________|___________                                                            
 |    |                    VP                                                         
 |    |     _______________|____________________                                       
 |    |    |                                    NP                                    
 |    |    |                        ____________|_______________________               
 |    |    |                       NP                                   |             
 |    |    |            ___________|__